In [1]:
%reload_ext autoreload
%autoreload 2

## Ejercicio 1: Clasificación con redes bayesianes y cálculo de probabilidades condicionales (parte 2) - Modelado

El objetivo de este notebook es modelar dos clasificadores que ayuden a predecir `churn` balanceando por un lado la capacidad predictiva de los mismos con su capacidad explicativa. Para ellos se partirá del dataset con las características más significativas ya trameadas obtenido como conclusión del notebook 010_churn_dataset_exploration y se realizará la selección de hiperparametros, al serialización y predicción del modelo utilizando diferentes métricas. Al ser un dataset desbalanceado se ha optado por técnicas de balanceo del dataset para el entrenamiento. Por otro lado la métrica escogida será AUC con el fin de mantener el equilibrio entre falsos positivos y negativos en cuanto a los clientes potenciales de fuga.

Se procede a cargar las librerías.

In [2]:
# # Import libraries
import pandas as pd
from pathlib import Path
from churn.preprocessing import load_data
from churn.paths import DATA_DIR, MODELS_DIR


import churn.config as cfg
from functools import partial
import joblib
from IPython.display import display, HTML
import plotly.offline as pyo
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours

# Import necessary functions from modelling.py
from churn.modelling import (
    split_features_and_label,
    train_tune_evaluate,
    calculate_classification_metrics,
    display_classification_results,
)

from churn.plot import draw_roc_curve
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold

from functools import partial

import pickle

Se importan los datos con las características más importantes ya trameadas.

In [3]:
# Define the file paths
train_path = Path(DATA_DIR / 'train_features_binned.parquet')
test_path = Path(DATA_DIR / 'test_features_binned.parquet')

# Load the raw data
train_features_binned = load_data(train_path) 
test_features_binned = load_data(test_path) 
# Display the first rows of the raw data
train_features_binned.head()

2024-09-04 03:40:58,641 - INFO - Data loaded from /Users/borja/Documents/Somniumrema/projects/ml/churn/data/train_features_binned.parquet
2024-09-04 03:40:58,645 - INFO - Data loaded from /Users/borja/Documents/Somniumrema/projects/ml/churn/data/test_features_binned.parquet


,churn,total_day_minutes,total_day_charge,total_eve_minutes,total_eve_calls,customer_service_rating,customer_happiness,customer_service_calls
0,0,6,5,1,0,3,1,0
1,0,6,3,1,1,1,0,0
2,0,7,0,1,2,3,5,0
3,1,7,3,1,2,3,0,5
4,0,7,5,1,1,3,2,3


Se emplea como estrategia de validación el 'repeated stratified cross validation' con 5 splits y 2 repeticiones.

In [4]:
# Define the cross-validation strategy
cv = RepeatedStratifiedKFold(n_splits=cfg.N_SPLITS, n_repeats=cfg.N_REPEATS, random_state=cfg.SEED)

Se definen los modelos de `Naive Bayes` y `SVC` con el fin de ajustar los hiperparametros de los mismos con las restricciones establecidas en cuanto a su elección. Adicionalmente se considera que el SVC tiene que tener los pesos balanceados y los valores máximos y mínimos de los hiperparámetros. En el caso de `Naive Bayes` se emplea una distribución de bernoulli con valores `alpha` entre 0.1 y 10 cubriendo un amplio espectro de suavizado de Laplace. en cuanto al coste de SVC los valore oscilan entre 0.1 y 100 indicando valores altos la penalización a la clasificacion erronea de observaciones.

In [5]:
# Define the models and their hyperparameter search spaces
models = {
    "Naive Bayes": (
        BernoulliNB,  # Use a lambda to include fixed parameters
        {
            "alpha": lambda trial: trial.suggest_float('alpha', cfg.BERNOULLI_LOWER_BOUND, cfg.BERNOULLI_UPPER_BOUND)
        }
    ),
    "SVM": (
        partial(SVC, probability=True, kernel='linear', weight='balanced'),
        {
            "C": lambda trial: trial.suggest_float('C', cfg.SVC_C_LOWER_BOUND, cfg.SCV_C_UPPER_BOUND, log=True) 
        }
    )
}

Se realiza el split en caracrerísticas y variable objetivo

In [6]:
# Split the features and label within the training and testing datasets
X_train, y_train = split_features_and_label(train_features_binned, 'churn')
X_test, y_test = split_features_and_label(test_features_binned, 'churn')

Se realiza el balanceo de clases para ello se sobremuestrear la clase minoritaria generando ejemplos sintéticos sin duplicar exactamente los  existentes y se eliminan observaciones redundantes en las clases. El fin es mejoraR la calidad del conjunto de datos balanceado al asegurarse de que los ejemplos que son muy similares a otras clases se eliminen, reduciendo la posibilidad de que el modelo aprenda relaciones incorrectas.

In [7]:
# Upsample the minority class using SMOTE and then apply EditedNearestNeighbours
smote_enn = SMOTEENN(smote=SMOTE(sampling_strategy='minority'), enn=EditedNearestNeighbours())
X_train_res, y_train_res = smote_enn.fit_resample(X_train, y_train)

# Combine the resampled features and labels into a single DataFrame
train_res = pd.concat([pd.DataFrame(X_train_res, columns=X_train.columns), pd.DataFrame(y_train_res, columns=['churn'])], axis=1)

Se realiza la optimización de hiperparámetros para ambos modelos

In [8]:
# # Train, tune, and evaluate the models
results = train_tune_evaluate(train_res, test_features_binned, models, cv=cv, n_trials=cfg.N_TRIALS)

2024-09-04 03:40:58,901 - INFO - Starting hyperparameter optimization for BernoulliNB...
Optimizing BernoulliNB: 100%|██████████| 50/50 [00:02<00:00, 23.43it/s]
2024-09-04 03:41:01,051 - INFO - Hyperparameter optimization for BernoulliNB completed.
2024-09-04 03:41:01,107 - INFO - Starting hyperparameter optimization for Pipeline...
Optimizing Pipeline: 100%|██████████| 50/50 [23:22<00:00, 28.05s/it]
2024-09-04 04:04:23,721 - INFO - Hyperparameter optimization for Pipeline completed.
2024-09-04 04:04:58,777 - INFO - Starting hyperparameter optimization for SVC...
Optimizing SVC: 100%|██████████| 50/50 [52:49<00:00, 63.39s/it]   
2024-09-04 04:57:48,493 - INFO - Hyperparameter optimization for SVC completed.


In [9]:
# Display the model results
for model_name, result in results.items():
    print(f"Model: {model_name}")
    for key, value in result.items():
        if key not in ['model', 'predictions_train', 'predictions_test', 'predictions_test_adjusted', 'predictions_train_proba', 'predictions_test_proba']:
            formatted_value = f"{value:.4f}" if isinstance(value, float) else value
            print(f"{key.replace('_', ' ').title()}: {formatted_value}")
    print()  # Print a newline

Model: Naive Bayes
Best Params: {'alpha': 7.518973772064895}
Roc Auc Cv: 0.8552
Roc Auc Train: 0.8553
Roc Auc Test: 0.8506
Threshold: 0.6270

Model: Scaler_SVM_pipeline
Best Params: {'svc__C': 12.920931205798915}
Roc Auc Cv: 0.9688
Roc Auc Train: 0.9689
Roc Auc Test: 0.9510
Threshold: 0.6446

Model: SVM
Best Params: {'C': 35.312550616059724}
Roc Auc Cv: 0.9688
Roc Auc Train: 0.9689
Roc Auc Test: 0.9510
Threshold: 0.6455



Los valores de la AUC son altos en los tres casos, siendo ligeramente mayores en el caso del SVC. El poder predictivo por lo tanto es maor en el caso del SV pero la capacidad explicativa del mismo es mucho mas compleja. En ambos casos se cambia el threshold y se optimiza de cara a la prediciones. Dicho valor asigna a una clase u otra en funciuón de la probabilidad estimada por el modelo. En el caso del SVC, el escalado de las variables carece de sentido por lo que se eliminó en sucesivas iteraciones aunque se deja aquí por su caracter explicativo. El motivo es que al escalar las variables se elimonan los tramos creaod entre ellas y las relaciones vistas con la V de Cramer desaparecen. Por otro lado se establece una vriable continua y estandarizada lo que mejora la capacidad de predicción del modelo lo cual en este caso se compensa. Al no se el objetivo de este análisis se procederá a evaluar las métricas únicamente con los modelos `Naive Bayes` y `SVC`.

In [ ]:
# Save each model to a pickle file
for model_name, result in results.items():
    model = result['model']
    model_filename = Path(MODELS_DIR / f'{model_name}_model.pkl')
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

In [10]:
for model_name, result in results.items():
    metrics = calculate_classification_metrics(train_res, test_features_binned, result['predictions_train'], result['predictions_test_adjusted'])
    display_classification_results(metrics, model_name)
    
    # Draw and display ROC curves
    fig_train = draw_roc_curve(train_res['churn'], result['predictions_train'], f'Receiver Operating Characteristic - {model_name} (Train)')
    display(HTML("<h3>ROC AUC (Train):</h3>"))
    pyo.iplot(fig_train)
    
    fig_test = draw_roc_curve(test_features_binned['churn'], result['predictions_test_proba'], f'Receiver Operating Characteristic - {model_name} (Test)')
    display(HTML("<h3>ROC AUC (Test):</h3>"))
    pyo.iplot(fig_test)
    
    display(HTML("<br>"))

- `Naive Bayes`

En el set de entrenamiento el f1 y la accuracy del modelo se encuentran cercanos a 0.8. En este caso en el test se incrementan los valores únicamente porque hay un número menor de observaciones. En lo que respecta a la matriz de confusión se puede observar que existen mal clasificadas 67 clientes que se fugan pero que no se identifican y 265 falsos positivos. El valor en train de AUC es de 0.79 a 0.85

- `SVC`

En el caso del SVC, al ACU, acccuracy se situan en valores superiores al 0.9. El f1 de la clase minoritria es superior situandose en valores por encima del 0.6. El número de falsos positivos y negativos es significativamente menor que en el caso del modelo de Naive Bayes (35 y 130) en test. En test los valores en este caso se mantienen

**Conclusión: Explicabilidad y Capacidad de Predicción**

Aunque desde el punto de vista predictivo el modelo de SVC es mejor que el de Naive Bayes, su explicabilidad únicamente radica en la aplicación de técnicas de LIME o SHAP considerandose como una 'caja negra' y no dando una explicación directa como en el caso de Niave Bayes. Adicionalmente computacionalmente es más complejo y requiere un tiempo mayor de entrenamiento por lo tanto puede plantear problemas en su puesta en producción. En el caso de Ninave Bayes, el modelo tiene pero capacidad predictiva pero mejor explicabilidad por lo que es fácil de interpretar al incorporarlo en el proceso de toma de decisiones. Se puede observar como diferentes características contribuyen al modelo directamente. Adicionalmente son más rápidos y simples de entrennar. Sin embargo presentan explicabilidad limitar en interacciones complejas entre variables así como una suposición de independencia entre las mismas que es muy fuerte y no se cumple en la mayoría de los casos.